In [1]:
import numpy as np
import cv2
import tensorflow as tf

定义计算

In [2]:
weight_decay = 1e-4


def relu(x, name='relu6'):
    return tf.nn.relu6(x, name)


def batch_norm(x, momentum=0.9, epsilon=1e-5, train=True, name='bn'):
    return tf.layers.batch_normalization(x,
                      momentum=momentum,
                      epsilon=epsilon,
                      scale=True,
                      training=train,
                      name=name)


def conv2d(input_, output_dim, k_h, k_w, d_h, d_w, stddev=0.02, name='conv2d', bias=False):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
              initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        if bias:
            biases = tf.get_variable('bias', [output_dim], initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)

        return conv


def conv2d_block(input, out_dim, k, s, is_train, name):
    with tf.name_scope(name), tf.variable_scope(name):
        net = conv2d(input, out_dim, k, k, s, s, name='conv2d')
        net = batch_norm(net, train=is_train, name='bn')
        net = relu(net)
        return net


def conv_1x1(input, output_dim, name, bias=False):
    with tf.name_scope(name):
        return conv2d(input, output_dim, 1, 1, 1, 1, stddev=0.02, name=name, bias=bias)


def pwise_block(input, output_dim, is_train, name, bias=False):
    with tf.name_scope(name), tf.variable_scope(name):
        out = conv_1x1(input, output_dim, bias=bias, name='pwb')
        out = batch_norm(out, train=is_train, name='bn')
        out = relu(out)
        return out


def dwise_conv(input, k_h=3, k_w=3, channel_multiplier= 1, strides=[1,1,1,1],
               padding='SAME', stddev=0.02, name='dwise_conv', bias=False):
    with tf.variable_scope(name):
        in_channel=input.get_shape().as_list()[-1]
        w = tf.get_variable('w', [k_h, k_w, in_channel, channel_multiplier],
                        regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                        initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.depthwise_conv2d(input, w, strides, padding, rate=None,name=None,data_format=None)
        if bias:
            biases = tf.get_variable('bias', [in_channel*channel_multiplier], initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)

        return conv


def res_block(input, expansion_ratio, output_dim, stride, is_train, name, bias=False, shortcut=True):
    with tf.name_scope(name), tf.variable_scope(name):
        # pw
        bottleneck_dim=round(expansion_ratio*input.get_shape().as_list()[-1])
        net = conv_1x1(input, bottleneck_dim, name='pw', bias=bias)
        net = batch_norm(net, train=is_train, name='pw_bn')
        net = relu(net)
        # dw
        net = dwise_conv(net, strides=[1, stride, stride, 1], name='dw', bias=bias)
        net = batch_norm(net, train=is_train, name='dw_bn')
        net = relu(net)
        # pw & linear
        net = conv_1x1(net, output_dim, name='pw_linear', bias=bias)
        net = batch_norm(net, train=is_train, name='pw_linear_bn')

        # element wise add, only for stride==1
        if shortcut and stride == 1:
            in_dim = int(input.get_shape().as_list()[-1])
            if in_dim != output_dim:
                ins = conv_1x1(input, output_dim, name='ex_dim')
                net = ins+net
            else:
                net = input+net

        return net


def separable_conv(input, k_size, output_dim, stride, pad='SAME', channel_multiplier=1, name='sep_conv', bias=False):
    with tf.name_scope(name), tf.variable_scope(name):
        in_channel = input.get_shape().as_list()[-1]
        dwise_filter = tf.get_variable('dw', [k_size, k_size, in_channel, channel_multiplier],
                  regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                  initializer=tf.truncated_normal_initializer(stddev=0.02))

        pwise_filter = tf.get_variable('pw', [1, 1, in_channel*channel_multiplier, output_dim],
                  regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                  initializer=tf.truncated_normal_initializer(stddev=0.02))
        strides = [1, stride, stride, 1]

        conv = tf.nn.separable_conv2d(input, dwise_filter, pwise_filter, strides, padding=pad, name=name)
        if bias:
            biases = tf.get_variable('bias', [output_dim], initializer=tf.constant_initializer(0.0))
            conv = tf.nn.bias_add(conv, biases)
        return conv


def global_avg(x):
    with tf.name_scope('global_avg'):
        net = tf.layers.average_pooling2d(x, x.get_shape()[1:-1], 1)
        return net


def flatten(x):
    # flattened = tf.reshape(input,[x.get_shape().as_list()[0], -1])  # or, tf.layers.flatten(x)
    return tf.contrib.layers.flatten(x)


def pad2d(inputs, pad=(0, 0), mode='CONSTANT'):
    paddings = [[0, 0], [pad[0], pad[0]], [pad[1], pad[1]], [0, 0]]
    net = tf.pad(inputs, paddings, mode=mode)
    return net

定义网络结构

In [3]:
def mobilenetv2(inputs, num_classes, is_train=True, reuse=False):

    exp = 6  # expansion ratio

    net = conv2d_block(inputs, 32, 3, 1, is_train, name='conv1_1')  # [64,128,96,32]

    net = res_block(net, 1, 16, 1, is_train, name='res1_1')  # [64,128,96,16]

    net = res_block(net, 1, 16, 2, is_train, name='res2_1')  # size/2 [64,64,48,16]

    net = res_block(net, exp, 32, 2, is_train, name='res3_1')  # size/4 [64,32,24,32]

    net = res_block(net, exp, 64, 2, is_train, name='res4_1')  # size/8 [64,16,12,64]

    net = res_block(net, exp, 128, 2, is_train, name='res5_1')  # size/16 [64,8,6,128]
    
    net = res_block(net, exp, 256, 2, is_train, name='res6_1')  # size/16 [64,8,6,128]

    net = pwise_block(net, 256, is_train, name='pw_conv7_1')  # [64,8,6,256]

    net = global_avg(net)  # [64,1,1,256]

    net = conv_1x1(net, num_classes, name='logits')  # [64,1,1,10]

    logits = flatten(net)  # [64,10]

    return logits

读取训练和验证数据

In [4]:
min_after_dequeue = 32
batch_size = 32
capacity = min_after_dequeue+3*batch_size

In [5]:
files = tf.train.match_filenames_once('E:/celeA/1220/data/data.tfrecords-*')

filename_queue = tf.train.string_input_producer(files,shuffle=True)

reader = tf.TFRecordReader()
_,serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.string)
    })

img = tf.decode_raw(features['image'],tf.float32)
image = tf.reshape(img, [128,96,3])
l = tf.decode_raw(features['label'],tf.float32)
label = tf.reshape(l, [42,2])

batch_x,batch_y = tf.train.shuffle_batch([image,label],batch_size=batch_size,capacity=capacity,min_after_dequeue=min_after_dequeue)

In [ ]:
vfiles = tf.train.match_filenames_once('E:/celeA/1220/data/test.tfrecords-*')
vfilename_queue = tf.train.string_input_producer(files,shuffle=False)

reader = tf.TFRecordReader()
_,vserialized_example = reader.read(vfilename_queue)
vfeatures = tf.parse_single_example(
    vserialized_example,
    features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.string)
    })

vimg = tf.decode_raw(vfeatures['image'],tf.float32)
vimage = tf.reshape(vimg, [128,96,3])
vl = tf.decode_raw(vfeatures['label'],tf.float32)
vlabel = tf.reshape(vl, [42,2])
vbatch_x,vbatch_y = tf.train.shuffle_batch([vimage,vlabel],batch_size=batch_size,capacity=capacity,min_after_dequeue=min_after_dequeue)

训练

In [7]:
w = 96
h = 128
c = 3
num_class = 84

n_step = 2001

model_path = r'E:/celeA/1220/model/model'
summary_path = r'E:/celeA/1220/log'

In [8]:
with tf.name_scope('input'):
    # 占位符
    x = tf.placeholder(tf.float32, shape=[None, h, w, c], name='x')
    y_ = tf.placeholder(tf.float32, shape=[None, 42, 2], name='y_')
    is_train = tf.placeholder(tf.bool)

logits = mobilenetv2(x, num_class, is_train)
pre = tf.reshape(logits, [-1, 42, 2])
res = tf.nn.softmax(pre,name='res')

b = tf.constant(value=1, dtype=tf.float32)
logits_eval = tf.multiply(pre, b, name='logits_eval')

global_step = tf.Variable(0)

with tf.name_scope('train'):
    with tf.name_scope('corss_entropy'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pre, labels=y_))
        tf.summary.scalar('corss_entropy', loss)
    with tf.name_scope('train_op'):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss, global_step=global_step)

with tf.name_scope('acc'):
    with tf.name_scope('correct_prediction'):
        max_idx_p = tf.argmax(res, 2)
        max_idx_l = tf.argmax(y_, 2)
        correct = tf.equal(max_idx_p, max_idx_l)
    with tf.name_scope('accuary'):
        acc = tf.reduce_mean(tf.cast(correct, tf.float32))
        tf.summary.scalar('accuary', acc)

merged = tf.summary.merge_all()
saver = tf.train.Saver(max_to_keep=10)

with tf.Session() as sess:
    # summary_writer
    train_writer = tf.summary.FileWriter(summary_path, tf.get_default_graph())
    # 变量初始化
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # 启动多线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    max_ac = 0
    # training
    for step in range(n_step):
        bx,by = sess.run([batch_x,batch_y])
        feed_dict = {x: bx, y_: by, is_train: True}
        train_sum, _, err, ac = sess.run([merged, train_op, loss, acc], feed_dict=feed_dict)
        print("step:{},train_loss:{},train_acc:{}".format(step, err, ac))
        train_writer.add_summary(train_sum, step)
            
        # validation
        vac = 0
        if step%50 == 0:
            for vv in range(3):
                vbx,vby = sess.run([vbatch_x,vbatch_y])
                feed_dict = {x: vbx, y_: vby, is_train: False}
                v_ac = sess.run(acc, feed_dict=feed_dict)
                vac += v_ac
            print("val_acc:{}".format(vac/3))    
            if vac>max_ac:
                max_ac = vac
                saver.save(sess, model_path, global_step=global_step)
                        
                

    train_writer.close()
    
    coord.request_stop()
    coord.join(threads)

step:0,train_loss:0.6751112937927246,train_acc:0.5803571343421936
val_acc:0.7425595124562582
step:1,train_loss:0.6305543184280396,train_acc:0.7485119104385376
step:2,train_loss:0.5829688310623169,train_acc:0.7924107313156128
step:3,train_loss:0.5380817651748657,train_acc:0.8035714030265808
step:4,train_loss:0.5159462094306946,train_acc:0.796875
step:5,train_loss:0.4957168698310852,train_acc:0.793154776096344
step:6,train_loss:0.4653405547142029,train_acc:0.8072916865348816
step:7,train_loss:0.48440298438072205,train_acc:0.784970223903656
step:8,train_loss:0.47303786873817444,train_acc:0.7909226417541504
step:9,train_loss:0.47282350063323975,train_acc:0.7909226417541504
step:10,train_loss:0.45030245184898376,train_acc:0.8013392686843872
step:11,train_loss:0.46433430910110474,train_acc:0.7864583134651184
step:12,train_loss:0.4563777446746826,train_acc:0.7834821343421936
step:13,train_loss:0.4601074457168579,train_acc:0.7916666865348816
step:14,train_loss:0.46199488639831543,train_acc:0.7

step:122,train_loss:0.30612075328826904,train_acc:0.8705357313156128
step:123,train_loss:0.3273783028125763,train_acc:0.867559552192688
step:124,train_loss:0.31610724329948425,train_acc:0.8623511791229248
step:125,train_loss:0.348629891872406,train_acc:0.855654776096344
step:126,train_loss:0.2901322841644287,train_acc:0.8809523582458496
step:127,train_loss:0.33531123399734497,train_acc:0.8504464030265808
step:128,train_loss:0.3365781307220459,train_acc:0.8645833134651184
step:129,train_loss:0.30431225895881653,train_acc:0.8653273582458496
step:130,train_loss:0.32821744680404663,train_acc:0.8526785969734192
step:131,train_loss:0.32523486018180847,train_acc:0.8616071343421936
step:132,train_loss:0.3319903612136841,train_acc:0.851934552192688
step:133,train_loss:0.323744535446167,train_acc:0.8653273582458496
step:134,train_loss:0.32884469628334045,train_acc:0.851934552192688
step:135,train_loss:0.330759197473526,train_acc:0.8616071343421936
step:136,train_loss:0.3440247178077698,train_acc

step:243,train_loss:0.25921717286109924,train_acc:0.8839285969734192
step:244,train_loss:0.27415451407432556,train_acc:0.8816964030265808
step:245,train_loss:0.2799224257469177,train_acc:0.8772321343421936
step:246,train_loss:0.2684113085269928,train_acc:0.8802083134651184
step:247,train_loss:0.26734262704849243,train_acc:0.8757440447807312
step:248,train_loss:0.2426847219467163,train_acc:0.9017857313156128
step:249,train_loss:0.2895931899547577,train_acc:0.8720238208770752
step:250,train_loss:0.27751049399375916,train_acc:0.8764880895614624
val_acc:0.8839285572369894
step:251,train_loss:0.26513856649398804,train_acc:0.8913690447807312
step:252,train_loss:0.23862527310848236,train_acc:0.9002976417541504
step:253,train_loss:0.2629919946193695,train_acc:0.8802083134651184
step:254,train_loss:0.24753643572330475,train_acc:0.8965773582458496
step:255,train_loss:0.2588609755039215,train_acc:0.8898809552192688
step:256,train_loss:0.257335901260376,train_acc:0.8913690447807312
step:257,train_

step:364,train_loss:0.19225402176380157,train_acc:0.921875
step:365,train_loss:0.21553079783916473,train_acc:0.9114583134651184
step:366,train_loss:0.21573588252067566,train_acc:0.9084821343421936
step:367,train_loss:0.2154703587293625,train_acc:0.9069940447807312
step:368,train_loss:0.2235533446073532,train_acc:0.9055059552192688
step:369,train_loss:0.21064095199108124,train_acc:0.9151785969734192
step:370,train_loss:0.19580155611038208,train_acc:0.9196428656578064
step:371,train_loss:0.2267788201570511,train_acc:0.9047619104385376
step:372,train_loss:0.20976795256137848,train_acc:0.914434552192688
step:373,train_loss:0.20124666392803192,train_acc:0.9248511791229248
step:374,train_loss:0.2112417221069336,train_acc:0.9055059552192688
step:375,train_loss:0.23053637146949768,train_acc:0.902529776096344
step:376,train_loss:0.2226959466934204,train_acc:0.9032738208770752
step:377,train_loss:0.2335834801197052,train_acc:0.9047619104385376
step:378,train_loss:0.21776239573955536,train_acc:0.

step:484,train_loss:0.17753519117832184,train_acc:0.9166666865348816
step:485,train_loss:0.1648958921432495,train_acc:0.933779776096344
step:486,train_loss:0.1696964055299759,train_acc:0.9345238208770752
step:487,train_loss:0.17493736743927002,train_acc:0.9270833134651184
step:488,train_loss:0.18844535946846008,train_acc:0.9203869104385376
step:489,train_loss:0.16717574000358582,train_acc:0.9315476417541504
step:490,train_loss:0.18042689561843872,train_acc:0.9211309552192688
step:491,train_loss:0.17382952570915222,train_acc:0.933779776096344
step:492,train_loss:0.17759323120117188,train_acc:0.930059552192688
step:493,train_loss:0.1787613481283188,train_acc:0.9308035969734192
step:494,train_loss:0.1744960993528366,train_acc:0.925595223903656
step:495,train_loss:0.17289979755878448,train_acc:0.9278273582458496
step:496,train_loss:0.1648693084716797,train_acc:0.930059552192688
step:497,train_loss:0.17050407826900482,train_acc:0.930059552192688
step:498,train_loss:0.16793020069599152,train

...

step:1432,train_loss:0.02306828647851944,train_acc:0.9970238208770752
step:1433,train_loss:0.01388230174779892,train_acc:0.9992559552192688
step:1434,train_loss:0.029169581830501556,train_acc:0.992559552192688
step:1435,train_loss:0.03366515040397644,train_acc:0.991815447807312
step:1436,train_loss:0.020151546224951744,train_acc:0.996279776096344
step:1437,train_loss:0.019535623490810394,train_acc:0.9985119104385376
step:1438,train_loss:0.025145672261714935,train_acc:0.9940476417541504
step:1439,train_loss:0.026058098301291466,train_acc:0.9947916865348816
step:1440,train_loss:0.021057628095149994,train_acc:0.9977678656578064
step:1441,train_loss:0.021664246916770935,train_acc:0.9970238208770752
step:1442,train_loss:0.017614472657442093,train_acc:0.9985119104385376
step:1443,train_loss:0.023425955325365067,train_acc:0.9955357313156128
step:1444,train_loss:0.016233021393418312,train_acc:0.9985119104385376
step:1445,train_loss:0.02874247170984745,train_acc:0.991815447807312
step:1446,trai

step:1549,train_loss:0.018229082226753235,train_acc:0.9970238208770752
step:1550,train_loss:0.018785377964377403,train_acc:0.9955357313156128
val_acc:0.9997519850730896
step:1551,train_loss:0.020109692588448524,train_acc:0.9970238208770752
step:1552,train_loss:0.028373729437589645,train_acc:0.9903273582458496
step:1553,train_loss:0.015559082850813866,train_acc:0.9985119104385376
step:1554,train_loss:0.023119712248444557,train_acc:0.992559552192688
step:1555,train_loss:0.025789115577936172,train_acc:0.9947916865348816
step:1556,train_loss:0.021068040281534195,train_acc:0.9955357313156128
step:1557,train_loss:0.020614206790924072,train_acc:0.9970238208770752
step:1558,train_loss:0.022938385605812073,train_acc:0.9947916865348816
step:1559,train_loss:0.0197361558675766,train_acc:0.996279776096344
step:1560,train_loss:0.016745300963521004,train_acc:0.9985119104385376
step:1561,train_loss:0.01822882890701294,train_acc:0.9970238208770752
step:1562,train_loss:0.016875561326742172,train_acc:0.9

step:1665,train_loss:0.01603863388299942,train_acc:0.9955357313156128
step:1666,train_loss:0.014630276709794998,train_acc:0.9977678656578064
step:1667,train_loss:0.015089582651853561,train_acc:0.9970238208770752
step:1668,train_loss:0.00930013321340084,train_acc:1.0
step:1669,train_loss:0.013643796555697918,train_acc:0.9985119104385376
step:1670,train_loss:0.020120084285736084,train_acc:0.9970238208770752
step:1671,train_loss:0.012728726491332054,train_acc:0.9992559552192688
step:1672,train_loss:0.020461896434426308,train_acc:0.9970238208770752
step:1673,train_loss:0.018060533329844475,train_acc:0.9970238208770752
step:1674,train_loss:0.013061873614788055,train_acc:1.0
step:1675,train_loss:0.018032129853963852,train_acc:0.9970238208770752
step:1676,train_loss:0.02022217959165573,train_acc:0.9947916865348816
step:1677,train_loss:0.01596381887793541,train_acc:0.9970238208770752
step:1678,train_loss:0.011782304383814335,train_acc:1.0
step:1679,train_loss:0.012011511251330376,train_acc:0.9

step:1783,train_loss:0.019558873027563095,train_acc:0.9940476417541504
step:1784,train_loss:0.018673868849873543,train_acc:0.9940476417541504
step:1785,train_loss:0.019704077392816544,train_acc:0.996279776096344
step:1786,train_loss:0.016718914732336998,train_acc:0.9970238208770752
step:1787,train_loss:0.015332406386733055,train_acc:0.9947916865348816
step:1788,train_loss:0.013912122696638107,train_acc:0.9985119104385376
step:1789,train_loss:0.013836230151355267,train_acc:0.9985119104385376
step:1790,train_loss:0.0119326738640666,train_acc:0.9970238208770752
step:1791,train_loss:0.012093739584088326,train_acc:0.9977678656578064
step:1792,train_loss:0.02107827737927437,train_acc:0.9947916865348816
step:1793,train_loss:0.01873374916613102,train_acc:0.9955357313156128
step:1794,train_loss:0.03465870022773743,train_acc:0.9895833134651184
step:1795,train_loss:0.019019385799765587,train_acc:0.9955357313156128
step:1796,train_loss:0.01895296387374401,train_acc:0.9955357313156128
step:1797,tra

step:1899,train_loss:0.021114900708198547,train_acc:0.9955357313156128
step:1900,train_loss:0.014865679666399956,train_acc:0.9970238208770752
val_acc:0.9985119104385376
step:1901,train_loss:0.014488719403743744,train_acc:0.9970238208770752
step:1902,train_loss:0.024409430101513863,train_acc:0.9933035969734192
step:1903,train_loss:0.02549808844923973,train_acc:0.991815447807312
step:1904,train_loss:0.011206558905541897,train_acc:0.9985119104385376
step:1905,train_loss:0.021417727693915367,train_acc:0.9947916865348816
step:1906,train_loss:0.018741462379693985,train_acc:0.9970238208770752
step:1907,train_loss:0.018639760091900826,train_acc:0.9970238208770752
step:1908,train_loss:0.023745210841298103,train_acc:0.992559552192688
step:1909,train_loss:0.013986608944833279,train_acc:0.9985119104385376
step:1910,train_loss:0.015286012552678585,train_acc:0.9985119104385376
step:1911,train_loss:0.032984279096126556,train_acc:0.9895833134651184
step:1912,train_loss:0.018961111083626747,train_acc:0